<a href="https://colab.research.google.com/github/AndreaMT05/Bussiness-intelligence/blob/main/Reto_2_Arbol_de_decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El analisis de decisión será en base a 7 variables socioeconomicas de los estudiantes que son admitidos en la universidad.

Descripción de las variables

- Edu_Parent1: Indica el nivel educativo alcanzado por el primer padre o tutor del solicitante.
- Edu_Parent2: Indica el nivel educativo alcanzado por el segundo padre o tutor del solicitante.
- Gender: Representa el género del solicitante, donde 0 corresponde a masculino y 1 a femenino.
- White: Variable binaria que indica si el solicitante se identifica como blanco (1) o no (0).
- Asian: Variable binaria que indica si el solicitante se identifica como asiático (1) o no (0).
- HSGPA: Promedio de calificaciones obtenido por el solicitante en la educación secundaria (High School GPA).
- SAT/ACT: Puntaje obtenido por el solicitante en los exámenes de admisión estandarizados SAT o ACT.
- Admitted_Pred: Indica la admisión del solicitante a la universidad, donde 1 significa admitido y 0 significa no admitido.

0. Cargamos las librerias de datos

In [ ]:
#Librerias
import numpy as np
import pandas as pd

#Librerias arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Cargamos la base de datos

In [ ]:
nxl= '/content/sample_data/2. CollegeAdmisions_Int_M.xlsx'
XDB= pd.read_excel(nxl, sheet_name=0)
XDB=XDB.dropna()
XDB.head(100)

#Cambiaremos la variable 'Gender'
XDB ['Gender'] = XDB['Gender'].map({'M':0,'F':1})

XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
YD=XDB[['Admitted']]

2. Contar # de personas admitidas por la universidad

In [ ]:
#cambio de variable 'admitted' el YES o NO por 1 o 0
XDB['Admitted']= XDB['Admitted'].map({'Yes':1,'No':0})

#Contamos el numero de datos de la base
total_datos=len(XDB)
print(f"El numero total de datos de la base es:{total_datos}")

#contamos numero de personas admitidas
admitidos =XDB['Admitted'].sum()
print(f"El numero total de datos de la base es: {admitidos}")

El numero total de datos de la base es:1528
El numero total de datos de la base es: 1303


3. Comencemos con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,YD)

#Realizar la grafica de arbol de decision
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']
dot_data=export_graphviz(mar, feature_names=ve, filled=True, rounded=True)
graph=graph_from_dot_data(dot_data)
graph.write_png('Reto#2. Arbol de decision.png')

True

Determinar numero de nodos para el arbol segun la estructura del modelo

In [ ]:
nodos_puros = sum(mar.tree_.impurity == 0.0)
print(f"Número de nodos puros en el árbol: {nodos_puros}")

Número de nodos puros en el árbol: 7


5. continuamos con el pronostico de estudiantes

In [ ]:
# Cargar la base de datos de pronóstico
XDB0 = pd.read_excel(nxl, sheet_name=1)
XDB0.head(100)

# Convertir 'Gender' a valores numéricos
XDB0['Gender'] = XDB0['Gender'].map({'M': 0, 'F': 1})

# Convertir 'Admitted' a valores numéricos
XDB0['Admitted'] = XDB0['Admitted'].map({'Yes': 1, 'No': 0})

# Seleccionar las variables de entrada y salida
XD0 = XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']].values
YD0 = XDB0[['Admitted']].values  # Esto es solo para referencia, no se usa en la predicción

# Realizar el pronóstico de admisión para todos los estudiantes
y_pred = mar.predict(XD0)

# Crear el informe en Excel con los resultados
df = pd.DataFrame(np.column_stack((XDB0.index, XD0, y_pred)))
df.columns = ['Solicitante', 'Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT', 'Admitted_Pred']

#Guardar en excel con resultados
df.to_excel("/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx")

print("El pronostico de admision ha sido guardado en '2. CollegeAdmisions_Int_M.xlsx")

El pronostico de admision ha sido guardado en '2. CollegeAdmisions_Int_M.xlsx


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


6. Evaluar rendimiento

In [ ]:
YDP=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(YD,YDP)
print("La matriz de confusión es: \n", cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

#Métricas de desempeño
EX=(VP+VN)/(VP+VN+FP+FN) #Exactitud
print('La exactitud del modelo es:\n',EX)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error:
print('La tasa de error del modelo es:\n',TE)

SE=VP/(VP+FN) #Sensibilidad
print('La sensibilidad del modelo es:\n',SE)

SP=VN/(VN+FP) #Especificidad
print('La especificidad del modelo es:\n',SP)

PR=VP/(VP+FP) #Precisión
print('La precisión del modelo es:\n',PR)

PRNeng=VN/(VN+FN) #Precisión Negativa
print('La precisión negativa del modelo es:\n',PRNeng)

La matriz de confusión es: 
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


Analisis

Todo el modelo en total tuvo 1528 registros en donde se observo que 1303 fueron admitidos los estudiantes y solo 225 no lo fueron.

Dentro de los verdaderos positivos se encuentran 1301 estudiantes admitidos y solo 20 no adminitidos que son los verdaderos negativos. De los estudiantes 205 en realidad no fueron admitidos aunque así fueron clasificados, es decir, falsos positivos; solo 2 admitidos fueron clasificados de forma errada como no admitidos (falsos negativos).

La mayoria de los valores estan por encima del 75% en el modelo, lo que permite ver el buen funcionamiento de este. La exactitud general del modelo es 86.45%, lo que indica un buen desempeño en la clasificación. La sensibilidad fue de 99.85% y muestra que identifica correctamente casi todos los admitidos, su precisión fue del 86.39% indica que la mayoría de los clasificados como admitidos realmente lo son. Sin embargo, la especificidad es solo del 8.89%, lo que sugiere que el modelo tiene dificultades para detectar los no admitidos, clasificando erróneamente a muchos de ellos. Esto se refleja en una tasa de error del 13.55%. La precisión negativa es del 90.91%, lo que significa que la mayoría de los clasificados como no admitidos lo fueron correctamente.

En general, es un modelo confiable para predecir admisiones pero necesita sus ajustes para terminar de identificar al 100% los estudiantes que son o no admitidos.